In [1]:
from dotenv import load_dotenv
from snowflake.snowpark.session import Session
import os
from snowflake.core import Root
from typing import List

C:\Users\Administrator\AppData\Local\Temp\ipykernel_29372\824325259.py:2: DeprecationWarning: Python Runtime 3.8 reached its End-Of-Life (EOL) on October 14, 2024, there will be no further bug fixes or security updates for this runtime. We recommend that you upgrade your existing Python 3.8 objects to Python 3.9, 3.10 or 3.11 before March 31, 2025. Please note that end of support does not impact execution, and you will still be able to update and invoke existing objects. However, they will be running on an unsupported runtime which will no longer be maintained or patched by the Snowflake team. For more details, please refer to https://docs.snowflake.com/en/developer-guide/python-runtime-support-policy.
  from snowflake.snowpark.session import Session


In [2]:
load_dotenv(override=True)

connection_params = {
  "account":  os.environ["SNOWFLAKE_ACCOUNT"],
  "user": os.environ["SNOWFLAKE_USER"],
  "password": os.environ["SNOWFLAKE_USER_PASSWORD"],
  "role": os.environ["SNOWFLAKE_ROLE"],
  "database": os.environ["SNOWFLAKE_DATABASE"],
  "schema": os.environ["SNOWFLAKE_SCHEMA"],
  "warehouse": os.environ["SNOWFLAKE_WAREHOUSE"],
}

snowpark_session = Session.builder.configs(connection_params).create()

In [3]:
class CortexSearchRetriever:

    def __init__(self, snowpark_session: Session, limit_to_retrieve: int = 4):
        self._snowpark_session = snowpark_session
        self._limit_to_retrieve = limit_to_retrieve

    def retrieve(self, query: str) -> List[str]:
        root = Root(self._snowpark_session)
        cortex_search_service = (
            root.databases[os.environ["SNOWFLAKE_DATABASE"]]
            .schemas[os.environ["SNOWFLAKE_SCHEMA"]]
            .cortex_search_services[os.environ["SNOWFLAKE_CORTEX_SEARCH_SERVICE"]]
        )
        resp = cortex_search_service.search(
            query=query,
            columns=["NAME", "INFORMATION"],
            limit=self._limit_to_retrieve,
        )

        if resp.results:
            return [curr["INFORMATION"] for curr in resp.results]
        else:
            return []

In [4]:
retriever = CortexSearchRetriever(snowpark_session=snowpark_session, limit_to_retrieve=4)

In [5]:
retrieved_context = retriever.retrieve(query="North Cascade")
print(retrieved_context)

['The Cascades are a mountain range that stretches some 700 miles (1100 km) from British Columbia through central Washington into Oregon.  By causing moist Pacific air to drop rain and dry out as it blows into the continent, the Cascades divide Washington state into a wet west and an arid east.  The Cascades are tall -- several are above 10,000\' (3000m) high, including Mount Baker in this region.  Because their valleys are low and the mountains quite steep, many mountains in the Cascades are visually quite striking.To the west from the Cascades to the coast are lowlands. Interstate 5 (I-5) runs north from Seattle to Vancouver, British Columbia. The Skagit Valley is fertile farmland, overflowing with tulips in the spring. Further north, Bellingham is the last US city before the Canadian border.The North Cascades region is made up of Chelan, Kittitas, Okanogan, Skagit, and Whatcom counties.  The term "North Cascades" sometimes means the mountain chain as far south as Mount Rainier.  The

In [6]:
print(len(retrieved_context))

4


In [7]:
snowpark_session.close()